In [1]:
from neo4j import GraphDatabase
import pickle
import pandas as pd
from collections import defaultdict

In [2]:
driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "lv23623600"))

## get user information
- input: user_id
- output: dict_table
    - keys: account_id
    - values: transactions

In [3]:
def get_user_transaction(user_id):
    with driver.session() as session:
        res = session.run("match (a:Account) -[:belongTo]-> (c:Customer { id : $id }) return a", id = user_id )
        all_account = []
        for record in res:
            for item in record:
                all_account.append(item["id"])
        
        user_transaction_table = defaultdict(list)
        
        # 转出
        for account_id in all_account:
            res = session.run("match (a:Account {id : $id}) -[r:transaction]-> (b:Account) return r,a,b",id = account_id)
            for record in res:
                user_transaction_table[account_id].append([record['a']['id'],record['b']['id'],record['r']['TR_TM'],record['r']['TR_NO'],record['r']['TR_AM'],record['r']['OPR_id'],-1])
            
        # 转入
        for account_id in all_account:
            res = session.run("match (a:Account {id : $id}) <-[r:transaction]- (b:Account) return r,a,b",id = account_id)
            for record in res:
                user_transaction_table[account_id].append([record['a']['id'],record['b']['id'],record['r']['TR_TM'],record['r']['TR_NO'],record['r']['TR_AM'],record['r']['OPR_id'],1])
        
        return user_transaction_table

In [4]:
user_transaction_table = get_user_transaction(39868635)

In [9]:
print(user_transaction_table[9628017867])

[[9628017867, 5230792217, '06-SEP-18', 'IB00230117000068', 663397, 'IB0023', -1], [9628017867, 9484428688, '04-SEP-18', 'IB00350117000008', 636854, 'IB0035', -1], [9628017867, 7417148055, '02-SEP-18', 'IB00370117000016', 25028, 'IB0037', -1], [9628017867, 1761934891, '01-SEP-18', 'IB00210117000076', 368084, 'IB0021', 1], [9628017867, 9115786377, '27-AUG-18', 'ED00330101000098', 827708, 'ED0033', 1], [9628017867, 9585954512, '04-SEP-18', 'IB00090117000004', 417541, 'IB0009', 1], [9628017867, 8716053091, '31-AUG-18', 'IB00250141000267', 69301, 'IB0025', 1], [9628017867, 3391910758, '15-AUG-18', 'IB00390141000347', 220831, 'IB0039', 1], [9628017867, 5201087596, '04-SEP-18', 'IB00310117000082', 978997, 'IB0031', 1], [9628017867, 5159874764, '30-AUG-18', 'YL00080138000571', 87506, 'YL0008', 1]]


## get card information
- input: card_id, user_infor_table
- output: all transaction for a card

In [137]:
def get_card_transaction(card_id,user_transaction_table):
    all_trans_for_an_card = user_transaction_table[card_id]
    card_transaction = pd.DataFrame(data = all_trans_for_an_card, index = range(len(all_trans_for_an_card)), columns = ["MY CARD","TO CARD","TRAN TIME","TRAN NUMBER", "TRAN AMOUNT","OPR ID","FLAG" ] )
    card_transaction["TRAN TIME"] = pd.to_datetime(card_transaction["TRAN TIME"])
    card_transaction = card_transaction.set_index("TRAN TIME").sort_index()
    return card_transaction

In [138]:
user_transaction_table = get_user_transaction(39868635)
card_transaction = get_card_transaction(9628017867,user_transaction_table)
card_transaction

,MY CARD,TO CARD,TRAN NUMBER,TRAN AMOUNT,OPR ID,FLAG
TRAN TIME,,,,,,
2018-08-15,9628017867,3391910758,IB00390141000347,220831,IB0039,1
2018-08-27,9628017867,9115786377,ED00330101000098,827708,ED0033,1
2018-08-30,9628017867,5159874764,YL00080138000571,87506,YL0008,1
2018-08-31,9628017867,8716053091,IB00250141000267,69301,IB0025,1
2018-09-01,9628017867,1761934891,IB00210117000076,368084,IB0021,1
2018-09-02,9628017867,7417148055,IB00370117000016,25028,IB0037,-1
2018-09-04,9628017867,9484428688,IB00350117000008,636854,IB0035,-1
2018-09-04,9628017867,9585954512,IB00090117000004,417541,IB0009,1
2018-09-04,9628017867,5201087596,IB00310117000082,978997,IB0031,1


## Analysis for an card
- 转入总金额
- 转出总金额
- 转入与转出的差值
- 基于时间的转入总金额
- 基于时间的转出总金额
- 基于时间的转入与转出的差值

In [166]:
def analysis_account(card_transaction, time1 = None, time2 = None):
    
    sum_in = None
    sum_out = None
    time_sum_in = None
    time_sum_out = None
    difference = None
    time_difference = None
    
    groups = card_transaction.groupby(["FLAG"])
    for name, group in groups:
        if name == -1:
            sum_out = group["TRAN AMOUNT"].sum()
        else:
            sum_in = group["TRAN AMOUNT"].sum()
        
        #if time1 and time2:
        group = group[time1:time2]
        if name == -1:
            time_sum_out = group["TRAN AMOUNT"].sum()
        else:
            time_sum_in = group["TRAN AMOUNT"].sum()
        

    difference = sum_in - sum_out
    time_difference = time_sum_in - time_sum_out
    
    return sum_in, sum_out, time_sum_out, time_sum_in, difference, time_difference

In [168]:
sum_in, sum_out, time_sum_out, time_sum_in, difference, time_difference = analysis_account(card_transaction, "2018-08-27", "2018-09-04")
print(sum_in)
print(sum_out)
print(time_sum_out)
print(time_sum_in)
print(difference)
print(time_difference)

2969968
1325279
661882
2749137
1644689
2087255


## get teller information

In [11]:
def get_teller_transaction(teller_id ):
    with driver.session() as session:
        res = session.run("match (t:Teller { id : $id }) -[:serve]-> (a:Account) return a", id = teller_id )
        all_account = []
        for record in res:
            for item in record:
                all_account.append(item["id"])
        
        teller_transaction = []
        
        # 转出
        for account_id in all_account:
            res = session.run("match (a:Account {id : $acc_id}) -[r:transaction]-> (b:Account) where r.OPR_id = $opr_id return r,a,b",acc_id = account_id, opr_id = teller_id )
            for record in res:
                teller_transaction.append([record['a']['id'],record['b']['id'],record['r']['TR_TM'],record['r']['TR_NO'],record['r']['TR_AM'],record['r']['OPR_id']])
        
        
        teller_transaction = pd.DataFrame(data = teller_transaction, index = range(len(teller_transaction)), columns = ["FROM CARD","TO CARD","TRAN TIME","TRAN NUMBER", "TRAN AMOUNT","OPR ID"] )
        teller_transaction["TRAN TIME"] = pd.to_datetime(teller_transaction["TRAN TIME"])
        teller_transaction = teller_transaction.set_index("TRAN TIME").sort_index()
        
        return teller_transaction

In [12]:
teller_id = "IB0037"
teller_transaction = get_teller_transaction(teller_id)

## analysis for teller
- daily amount
- time interval transaction
- time interval amount
- account transaction
- major account

In [182]:
teller_transaction.head()

,FROM CARD,TO CARD,TRAN NUMBER,TRAN AMOUNT,OPR ID
TRAN TIME,,,,,
2018-08-15,8897675198,3703409130,IB00370117000006,762229,IB0037
2018-08-15,2205064882,4214662502,IB00370117000069,544656,IB0037
2018-08-15,1494869472,1497866091,IB00370117000068,224184,IB0037
2018-08-15,3317718363,3933450537,IB00370117000028,355465,IB0037
2018-08-15,2321935333,3523918870,IB00370117000026,817189,IB0037


In [26]:
def analysis_teller(teller_transaction, time1 = None, time2 = None):
    daily_amount = None
    time_interval_transaction = None
    time_interval_amount = None
    card_transaction = None
    major_account = None
    max_value = None
    
    if time1 and time2:    
        time_interval_transaction = teller_transaction[time1:time2]
        time_interval_amount = time_interval_transaction["TRAN AMOUNT"].sum()

    daily_amount = teller_transaction.groupby("TRAN TIME")["TRAN AMOUNT"].sum()
    card_transaction = teller_transaction.groupby("FROM CARD")["TRAN AMOUNT"].sum()
    major_account,max_value = card_transaction.idxmax(),card_transaction.max()
    
    
    return daily_amount,time_interval_transaction,time_interval_amount,card_transaction,(major_account,max_value)
    
daily_amount,time_interval_transaction,time_interval_amount,card_transaction, major = analysis_teller(teller_transaction, time1 = "2018-08-16", time2 = "2018-08-18")  

In [49]:
[[ str(item) for item in line]  for line in daily_amount.reset_index().values.tolist()]

[['2018-08-15 00:00:00', '5541678'],
 ['2018-08-16 00:00:00', '5550482'],
 ['2018-08-17 00:00:00', '3271752'],
 ['2018-08-18 00:00:00', '4554159'],
 ['2018-08-19 00:00:00', '6387881'],
 ['2018-08-20 00:00:00', '7812532'],
 ['2018-08-21 00:00:00', '3911297'],
 ['2018-08-22 00:00:00', '7433995'],
 ['2018-08-23 00:00:00', '9952583'],
 ['2018-08-24 00:00:00', '10260241'],
 ['2018-08-25 00:00:00', '11486662'],
 ['2018-08-26 00:00:00', '8172498'],
 ['2018-08-27 00:00:00', '14422007'],
 ['2018-08-28 00:00:00', '11323408'],
 ['2018-08-29 00:00:00', '20609951'],
 ['2018-08-30 00:00:00', '27751773'],
 ['2018-08-31 00:00:00', '25074929'],
 ['2018-09-01 00:00:00', '14573179'],
 ['2018-09-02 00:00:00', '17033254'],
 ['2018-09-03 00:00:00', '21993054'],
 ['2018-09-04 00:00:00', '22344893'],
 ['2018-09-05 00:00:00', '15769191'],
 ['2018-09-06 00:00:00', '11604388'],
 ['2018-09-07 00:00:00', '8122586'],
 ['2018-09-08 00:00:00', '4911167'],
 ['2018-09-09 00:00:00', '7116237'],
 ['2018-09-10 00:00:00', 